In [7]:
import sqlite3
import pandas as pd

def execute_sql_file(sql_file, db_name = "../travel_db.sqlite"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    with open(sql_file, 'r') as f:
        sql_script = f.read()

    try:
        cursor.execute(sql_script)
        rows = cursor.fetchall()
        # Get column names from cursor description
        if cursor.description:
            columns = [desc[0] for desc in cursor.description]
            df = pd.DataFrame(rows, columns=columns)
            return df
        else:
            print("Query executed successfully but returned no data.")
            return None
    except sqlite3.Error as e:
        print(f"Error executing statement: {e}")

    conn.commit()
    conn.close()


## 2b. I am Laura, 31, from Madrid. Currently visiting Warsaw in Poland. I don&#8217;t like my hotel. Show me hotels recommended by other people in desired price.


In [ ]:
%%cypher
MATCH (hotel:PlaceToSleep)-[IS_LOCATED_IN]->(:Place{name:'Warszawa'}),
(client:Person)-[:WENT_FOR]->(:Trip)-[stay:STAYED_AT]->hotel
WITH DISTINCT hotel.name as hotel, hotel.website as website, collect(stay) as stays
WITH hotel, website, (reduce(s = 0 , x IN stays | s + x.avg_price_per_night))/size(stays) as avg_price
WHERE avg_price<200
RETURN hotel, website, avg_price
ORDER BY avg_price

In [ ]:
execute_sql_file("hotel_avg_price_query.sql")

,hotel,website,avg_price
0,Villa Jeziorki 71,http://www.villajeziorki.pl,60.0
1,Mazowiecki,http://www.hotelmazowiecki.pl,112.0


## 1c. I am Laura, 31, from Madrid. I need an inspiration for a long travel. I want to see as many places as possible. Show me travels of other people.


In [ ]:
%%cypher
MATCH (:Trip)-[:IS_PART_OF]->(longTrip:Trip)-[:STARTS_FROM]->(start_place:Place)
WITH DISTINCT longTrip, start_place.name as start_place
MATCH (city:Place)<-[to:TO]-(shortTrip:Trip)-[part:IS_PART_OF]->(longTrip)
WITH longTrip.name as tripName, start_place, city.name as name, part.order_no as order_no, to.transportation as by
ORDER BY order_no
WITH tripName, start_place, collect({order_no:order_no, to:name, by:by}) as cities
WITH tripName, start_place, cities, size(cities) as nbrOfCities
RETURN tripName, start_place, cities
ORDER BY nbrOfCities DESC

In [9]:
execute_sql_file("long_trip_cities_query.sql")


,tripname,start_place,cities,nbrofcities
0,My trip around Europe,Warszawa,"{""order_no"":1,""to"":""Rome"",""by"":""plane""},{""orde...",15
1,My trip to USA,Madrid,"{""order_no"":1,""to"":""Chicago"",""by"":""plane""},{""o...",13
2,My trip around Poland,Rome,"{""order_no"":1,""to"":""Warszawa"",""by"":""plane""},{""...",7
3,My holidays to USA,London,"{""order_no"":1,""to"":""San Francisco"",""by"":""plane...",5
4,My trip around Poland,Chicago,"{""order_no"":1,""to"":""Warszawa"",""by"":""plane""},{""...",5
5,My trip around Poland,Lisboa,"{""order_no"":1,""to"":""Warszawa"",""by"":""plane""},{""...",5
6,My trip around Poland,Nice,"{""order_no"":1,""to"":""Poznan"",""by"":""plane""},{""or...",4
7,Weekend in Barcelona,Madrid,"{""order_no"":1,""to"":""Barcelona"",""by"":""plane""},{...",4
8,Weekend in London,Madrid,"{""order_no"":1,""to"":""London"",""by"":""plane""},{""or...",2


In [ ]:
%%cypher
MATCH (weekend:Trip{duration:2})-[:STARTS_FROM]->(madrid:Place{name:'Madrid'}),
(trip:Trip)-[:IS_PART_OF]->(weekend),
(trip)-[:TO]->(place:Place)
WHERE place.name <> 'Madrid'
WITH place.name as placeName, count(place) as counts
RETURN placeName
ORDER BY counts DESC

In [10]:
execute_sql_file("popular_madrid_destinations.sql")

,placename,counts
0,Barcelona,2
1,London,1


## 2a. I am Laura, 31, from Madrid. Currently visiting Warsaw in Poland. What restaurants in Warsaw are mostly recommended by people in my age?



In [ ]:
%%cypher
MATCH (restaurant:Sustenance)-[IS_LOCATED_IN]->(:Place{name:'Warszawa'}),
(client:Person)-[:WENT_FOR]->(:Trip)-[meal:WENT_TO]->restaurant
WHERE client.age>25 AND client.age<36
WITH DISTINCT restaurant.name as resto, collect(meal) as meals
WITH resto, (reduce(s = 0 , x IN meals | s + x.rate))/size(meals) as avg_rate
RETURN resto, avg_rate
ORDER BY avg_rate DESC

In [11]:
execute_sql_file("restaurant_avg_rate_query.sql")

,resto,avg_rate
0,Pierogarnia Zapiecek,5.0
1,Literatka,5.0


## 1b. I am Laura, 31, from Madrid. I am planning to go to USA for one month. I want to see as many places as possible. Show me how people travel there.


In [ ]:
%%cypher
MATCH (shortTrip:Trip)-[:TO]->(:Place)-[:BELONGS_TO*]->(:Country{code:'US'}),
(shortTrip)-[:IS_PART_OF]->(usaTrip:Trip)-[:STARTS_FROM]->(start_place:Place)
WHERE usaTrip.duration<32
WITH DISTINCT usaTrip, start_place.name as start_place
MATCH (:Country{code:'US'})<-[:BELONGS_TO*]-(city:Place)<-[to:TO]-(shortTrip:Trip)-[part:IS_PART_OF]->(usaTrip)
WITH usaTrip.name as tripName, start_place, city.name as name, part.order_no as order_no, to.transportation as by
ORDER BY order_no
WITH tripName, start_place, collect({order_no:order_no, to:name, by:by}) as cities
WITH tripName, start_place, cities, size(cities) as nbrOfCities
RETURN tripName, start_place, cities
ORDER BY nbrOfCities DESC

In [12]:
execute_sql_file("usa_trips_query.sql")

,tripName,start_place,cities,nbrOfCities
0,My trip to USA,Madrid,"{""order_no"":1,""to"":""Chicago"",""by"":""plane""},{""o...",12
1,My holidays to USA,London,"{""order_no"":1,""to"":""San Francisco"",""by"":""plane...",4


## 2c. I am Laura, 31, from Madrid. Currently visiting Warsaw in Poland. I want to spend more time in Poland than I planned previously. Where can I go next?


In [ ]:
%%cypher
MATCH (warsawTrip:Trip)-[:TO]->(place:Place{name:'Warszawa'}),
(warsawTrip)-[warsawPart:IS_PART_OF]->(longTrip:Trip),
(previousPlace:Place)<-[:TO]-(previousTrip)-[previousPart:IS_PART_OF]->longTrip,
(place)-[:BELONGS_TO*]->(:Country{name:'Poland'})<-[BELONGS_TO]-(previousPlace)
WHERE previousPart.order_no = warsawPart.order_no -1
RETURN previousPlace.name as place
UNION
MATCH (warsawTrip:Trip)-[:TO]->(place:Place{name:'Warszawa'}),
(warsawTrip)-[warsawPart:IS_PART_OF]->(longTrip:Trip),
(nextPlace:Place)<-[:TO]-(nextTrip)-[nextPart:IS_PART_OF]->longTrip,
(place)-[:BELONGS_TO*]->(:Country{name:'Poland'})<-[BELONGS_TO]-(nextPlace)
WHERE nextPart.order_no = warsawPart.order_no +1
RETURN nextPlace.name as place

In [14]:
execute_sql_file("warsaw_neighboring_places_query.sql")

,place
0,Krakow
1,Poznan
2,Torun
3,Zakopane
